In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error
from statsmodels.tsa.deterministic import DeterministicProcess

In [2]:
# path to the dataset in Kaggle's notebook
path = '../input/store-sales-time-series-forecasting/'

### 1. Compute Moving Average of Oil Prices

In [3]:
# read oil price
data_oil = pd.read_csv(path + 'oil.csv', parse_dates=['date'], infer_datetime_format=True, index_col='date')

########################################################################################################################
# TODO: compute data_oil['ma_oil'] as the moving average of data_oil['dcoilwtico'] with window size 7
# Hint: check the documentation of .rolling() method of pandas.DataFrame
########################################################################################################################
data_oil['ma_oil'] = data_oil.dcoilwtico.rolling(window=7, center=False).mean()

# Create continguous moving average of oil prices
calendar = pd.DataFrame(index=pd.date_range('2013-01-01', '2017-08-31'))

########################################################################################################################
# TODO 1: merge two DataFrame instances (data_oil and calendar) such that the merged instances has the same indexes
# as calendar.
# TODO 2: replace each NaN in data_oil['ma_oil'] by the first non-null value before it.
# Hint: check the documentation of .merge() and .fillna() methods of pandas.DataFrame
########################################################################################################################
calendar = calendar.join(data_oil)
calendar.ma_oil.fillna(method='ffill', inplace=True)

calendar.head(15) # display some entries of calendar

,dcoilwtico,ma_oil
2013-01-01,NaN,NaN
2013-01-02,93.14,NaN
2013-01-03,92.97,NaN
2013-01-04,93.12,NaN
2013-01-05,NaN,NaN
2013-01-06,NaN,NaN
2013-01-07,93.20,NaN
2013-01-08,93.21,NaN
2013-01-09,93.08,NaN
2013-01-10,93.81,93.218571


### 2. Create Workday Feature

In [4]:
########################################################################################################################
# TODO: create a True/False feature calendar['wd'] to indicate whether each date is a workday (Monday-Friday) or not.
# Hint: check documentation of pandas.DatetimeIndex.dayofweek
########################################################################################################################
calendar['wd'] = calendar.index.dayofweek <= 4

calendar.head(15) # display some entries of calendar

,dcoilwtico,ma_oil,wd
2013-01-01,NaN,NaN,True
2013-01-02,93.14,NaN,True
2013-01-03,92.97,NaN,True
2013-01-04,93.12,NaN,True
2013-01-05,NaN,NaN,False
2013-01-06,NaN,NaN,False
2013-01-07,93.20,NaN,True
2013-01-08,93.21,NaN,True
2013-01-09,93.08,NaN,True
2013-01-10,93.81,93.218571,True


### 3. Read Train and Test Data

In [5]:
df_train = pd.read_csv(path + 'train.csv',
                       usecols=['store_nbr', 'family', 'date', 'sales'],
                       dtype={'store_nbr': 'category', 'family': 'category', 'sales': 'float32'},
                       parse_dates=['date'], infer_datetime_format=True)

df_train.date = df_train.date.dt.to_period('D')
df_train = df_train.set_index(['store_nbr', 'family', 'date']).sort_index()

df_train.head(15) # display some entries of the training data

sales
store_nbr family     date             
1         AUTOMOTIVE 2013-01-01    0.0
                     2013-01-02    2.0
                     2013-01-03    3.0
                     2013-01-04    3.0
                     2013-01-05    5.0
                     2013-01-06    2.0
                     2013-01-07    0.0
                     2013-01-08    2.0
                     2013-01-09    2.0
                     2013-01-10    2.0
                     2013-01-11    3.0
                     2013-01-12    2.0
                     2013-01-13    2.0
                     2013-01-14    2.0
                     2013-01-15    1.0

In [6]:
df_test = pd.read_csv(path + 'test.csv',
                      usecols=['store_nbr', 'family', 'date'],
                      dtype={'store_nbr': 'category', 'family': 'category'},
                      parse_dates=['date'], infer_datetime_format=True)

df_test.date = df_test.date.dt.to_period('D')
df_test = df_test.set_index(['store_nbr', 'family', 'date']).sort_index()

df_test.head(15) # display some entries of the testing data

Empty DataFrame
Columns: []
Index: [(1, AUTOMOTIVE, 2017-08-16), (1, AUTOMOTIVE, 2017-08-17), (1, AUTOMOTIVE, 2017-08-18), (1, AUTOMOTIVE, 2017-08-19), (1, AUTOMOTIVE, 2017-08-20), (1, AUTOMOTIVE, 2017-08-21), (1, AUTOMOTIVE, 2017-08-22), (1, AUTOMOTIVE, 2017-08-23), (1, AUTOMOTIVE, 2017-08-24), (1, AUTOMOTIVE, 2017-08-25), (1, AUTOMOTIVE, 2017-08-26), (1, AUTOMOTIVE, 2017-08-27), (1, AUTOMOTIVE, 2017-08-28), (1, AUTOMOTIVE, 2017-08-29), (1, AUTOMOTIVE, 2017-08-30)]

In [7]:
# set the range of data used in training
sdate = '2017-04-01'
edate = '2017-08-15'

# we will train a model that takes feature of a date as input and predicts the sales for each store and family of goods on that date.
y = df_train.unstack(['store_nbr', 'family']).loc[sdate:edate]


########################################################################################################################
# TODO: create the trend feature X: the value for sdate is 1, the value for the next day of sdate is 2, etc.
# Hint: check the documentation of DeterministicProcess, or this tutorial: https://www.kaggle.com/code/ryanholbrook/trend.
########################################################################################################################
train_dp = DeterministicProcess(
    index=pd.date_range(sdate, edate),
    order=1,
    drop=True
)
X = train_dp.in_sample()

# Extentions
X['oil']  = calendar.loc[sdate:edate]['ma_oil'].values
X['wd']   = calendar.loc[sdate:edate]['wd'].values

X.head(15)

,trend,oil,wd
2017-04-01,1.0,48.570000,False
2017-04-02,2.0,48.570000,False
2017-04-03,3.0,49.034286,True
2017-04-04,4.0,49.561429,True
2017-04-05,5.0,50.150000,True
2017-04-06,6.0,50.625714,True
2017-04-07,7.0,51.022857,True
2017-04-08,8.0,51.022857,False
2017-04-09,9.0,51.022857,False
2017-04-10,10.0,51.417143,True


### 4. Train Model!

In [8]:
model = LinearRegression()
model.fit(X, y)
y_pred = pd.DataFrame(model.predict(X), index=X.index, columns=y.columns)

In [9]:
# Results on the training set

y_pred   = y_pred.stack(['store_nbr', 'family']).reset_index()
y_target = y.stack(['store_nbr', 'family']).reset_index().copy()

y_target['sales_pred'] = y_pred['sales'].clip(0.) # Sales should be >= 0

########################################################################################################################
# TODO: show the training loss for each type of product.
# Hint: check the documentation of DataFrame.groupby() and GroupBy.apply().
########################################################################################################################
# y_target.groupby(by=['family']).head(30)

In [10]:
y_target.groupby('family').apply(lambda df: mean_squared_log_error(y_true=df.sales, y_pred=df.sales_pred))

family
AUTOMOTIVE                    0.275179
BABY CARE                     0.070663
BEAUTY                        0.286778
BEVERAGES                     0.205483
BOOKS                         0.028720
BREAD/BAKERY                  0.136432
CELEBRATION                   0.339573
CLEANING                      0.215838
DAIRY                         0.148766
DELI                          0.121253
EGGS                          0.203725
FROZEN FOODS                  0.183883
GROCERY I                     0.216049
GROCERY II                    0.373721
HARDWARE                      0.291368
HOME AND KITCHEN I            0.278903
HOME AND KITCHEN II           0.233429
HOME APPLIANCES               0.162268
HOME CARE                     0.132638
LADIESWEAR                    0.278059
LAWN AND GARDEN               0.300818
LINGERIE                      0.422315
LIQUOR,WINE,BEER              0.790037
MAGAZINES                     0.274707
MEATS                         0.175813
PERSONAL CARE     

The model has the most difficulty in predicting school and office supplies. This is most likely due to its seasonal spikes that we do not really account for with our features.

In [11]:
# Test predictions

stest = '2017-08-16'
etest = '2017-08-31'

########################################################################################################################
# TODO: create the feature matrix of test data.
# Hint: check the documentation of DeterministicProcess.
########################################################################################################################
X_test = train_dp.out_of_sample(len(pd.date_range(stest, etest)))

# Extentions...
X_test['oil']  = calendar.loc[stest:etest]['ma_oil'].values
X_test['wd']   = calendar.loc[stest:etest]['wd'].values

print(X_test)

sales_pred = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=y.columns)
sales_pred = sales_pred.stack(['store_nbr', 'family'])

sales_pred[sales_pred < 0] = 0. # Sales should be >= 0

            trend        oil     wd
2017-08-16  138.0  48.281429   True
2017-08-17  139.0  47.995714   True
2017-08-18  140.0  47.852857   True
2017-08-19  141.0  47.852857  False
2017-08-20  142.0  47.852857  False
2017-08-21  143.0  47.688571   True
2017-08-22  144.0  47.522857   True
2017-08-23  145.0  47.645714   True
2017-08-24  146.0  47.598571   True
2017-08-25  147.0  47.720000   True
2017-08-26  148.0  47.720000  False
2017-08-27  149.0  47.720000  False
2017-08-28  150.0  47.624286   True
2017-08-29  151.0  47.320000   True
2017-08-30  152.0  47.115714   True
2017-08-31  153.0  47.060000   True


In [12]:
# Create submission

df_sub = pd.read_csv(path + 'sample_submission.csv', index_col='id')
df_sub.sales = sales_pred.values
df_sub.to_csv('submission.csv', index=True)